In [1]:
import re
import nltk
import string
from nltk.cluster.util import cosine_distance
import networkx as nx

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diham\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diham\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
stop_words = nltk.corpus.stopwords.words('portuguese')

In [4]:
def preprocessamento(texto):
  texto_formatado = texto.lower()
  tokens = []
  for token in nltk.word_tokenize(texto_formatado):
    tokens.append(token)

  tokens = [palavra for palavra in tokens if palavra not in stop_words and palavra not in string.punctuation]
  texto_formatado = ' '.join([str(elemento) for elemento in tokens if not elemento.isdigit()])

  return texto_formatado

In [6]:
texto_original = """A inteligência artificial é a inteligência similar à humana máquinas. 
                    Definem como o estudo de agente artificial com inteligência. 
                    Ciência e engenharia de produzir máquinas com inteligência. 
                    Resolver problemas e possuir inteligência. 
                    Relacionada ao comportamento inteligente. 
                    Construção de máquinas para raciocinar. 
                    Aprender com os erros e acertos. 
                    Inteligência artificial é raciocinar nas situações do cotidiano."""

In [8]:
sent_originais = [sent for sent in nltk.sent_tokenize(texto_original)]
sent_formatada = [preprocessamento(sent_original) for sent_original in sent_originais]

In [9]:
sent_originais

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Definem como o estudo de agente artificial com inteligência.',
 'Ciência e engenharia de produzir máquinas com inteligência.',
 'Resolver problemas e possuir inteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de máquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.']

In [10]:
sent_formatada

['inteligência artificial inteligência similar humana máquinas',
 'definem estudo agente artificial inteligência',
 'ciência engenharia produzir máquinas inteligência',
 'resolver problemas possuir inteligência',
 'relacionada comportamento inteligente',
 'construção máquinas raciocinar',
 'aprender erros acertos',
 'inteligência artificial raciocinar situações cotidiano']

In [27]:
def calcula_similaridade_sentencas(sentenca1, sentenca2):
    from collections import Counter
    from scipy.spatial.distance import cosine
    
    palavras1 = nltk.word_tokenize(sentenca1)
    palavras2 = nltk.word_tokenize(sentenca2)

    todas_palavras = set(palavras1) | set(palavras2)

    contador1 = Counter(palavras1)
    contador2 = Counter(palavras2)

    vetor1 = [contador1[palavra] for palavra in todas_palavras]
    vetor2 = [contador2[palavra] for palavra in todas_palavras]

    return 1 - cosine(vetor1, vetor2)

In [34]:
calcula_similaridade_sentencas(sent_formatada[0], sent_formatada[1])

0.4743416490252569

In [41]:
import numpy as np

def calcula_matriz_similaridade(sentencas):
    num_sentencas = len(sentencas)
    matriz_similaridade = np.zeros((num_sentencas, num_sentencas))

    for i in range(num_sentencas):
        for j in range(i + 1, num_sentencas):
            matriz_similaridade[i][j] = calcula_similaridade_sentencas(sentencas[i], sentencas[j])
            matriz_similaridade[j][i] = matriz_similaridade[i][j]

    return matriz_similaridade

In [42]:
calcula_matriz_similaridade(sent_formatada)

array([[0.        , 0.47434165, 0.47434165, 0.35355339, 0.        ,
        0.20412415, 0.        , 0.47434165],
       [0.47434165, 0.        , 0.2       , 0.2236068 , 0.        ,
        0.        , 0.        , 0.4       ],
       [0.47434165, 0.2       , 0.        , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.2       ],
       [0.35355339, 0.2236068 , 0.2236068 , 0.        , 0.        ,
        0.        , 0.        , 0.2236068 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.20412415, 0.        , 0.25819889, 0.        , 0.        ,
        0.        , 0.        , 0.25819889],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.47434165, 0.4       , 0.2       , 0.2236068 , 0.        ,
        0.25819889, 0.        , 0.        ]])

In [43]:
def sumarizar(texto, quantidade_sentencas):
    sentencas_originais = nltk.sent_tokenize(texto)
    sentencas_formatadas = [preprocessamento(sentenca_original) for sentenca_original in sentencas_originais]
    
    matriz_similaridade = calcula_matriz_similaridade(sentencas_formatadas)
    grafo_similaridade = nx.from_numpy_array(matriz_similaridade)
    
    notas = nx.pagerank(grafo_similaridade)
    notas_ordenadas = sorted(((notas[i], nota) for i, nota in enumerate(sentencas_originais)), reverse=True)
    
    melhores_sentencas = [nota for (_, nota) in notas_ordenadas[:quantidade_sentencas]]
  
    return sentencas_originais, melhores_sentencas, notas_ordenadas

In [44]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(texto_original, 3)

In [45]:
sentencas_originais

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Definem como o estudo de agente artificial com inteligência.',
 'Ciência e engenharia de produzir máquinas com inteligência.',
 'Resolver problemas e possuir inteligência.',
 'Relacionada ao comportamento inteligente.',
 'Construção de máquinas para raciocinar.',
 'Aprender com os erros e acertos.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.']

In [48]:
melhores_sentencas

['A inteligência artificial é a inteligência similar à humana máquinas.',
 'Inteligência artificial é raciocinar nas situações do cotidiano.',
 'Ciência e engenharia de produzir máquinas com inteligência.']

In [49]:
notas_sentencas

[(0.2282092404017801,
  'A inteligência artificial é a inteligência similar à humana máquinas.'),
 (0.1839190802151221,
  'Inteligência artificial é raciocinar nas situações do cotidiano.'),
 (0.16331914507834966,
  'Ciência e engenharia de produzir máquinas com inteligência.'),
 (0.15437170333277767,
  'Definem como o estudo de agente artificial com inteligência.'),
 (0.12639273963873288, 'Resolver problemas e possuir inteligência.'),
 (0.09616903563964786, 'Construção de máquinas para raciocinar.'),
 (0.023809527846794958, 'Relacionada ao comportamento inteligente.'),
 (0.023809527846794958, 'Aprender com os erros e acertos.')]

In [52]:
def visualiza_resumo(titulo, lista_sentencas, melhores_sentencas):
  from IPython.core.display import HTML
  texto = ''

  display(HTML(f'<h1>Resumo do texto - {titulo}</h1>'))
  for i in lista_sentencas:
    if i in melhores_sentencas:
      texto += str(i).replace(i, f"<mark>{i}</mark>")
    else:
      texto += i
  display(HTML(f""" {texto} """))

In [53]:
visualiza_resumo('Teste', sentencas_originais, melhores_sentencas)

In [55]:
from goose3 import Goose
g = Goose()
url = 'https://iaexpert.academy/2020/11/09/ia-preve-resultado-das-eleicoes-americanas/'
artigo = g.extract(url)

In [60]:
sentencas_originais, melhores_sentencas, notas_sentencas = sumarizar(artigo.cleaned_text, 5)

In [61]:
sentencas_originais

['Nas eleições presidenciais americanas de 2016, a maioria das predições apontavam para a vitória de Hillary Clinton.',
 'Entretanto, a história nos mostrou o resultado oposto, e Donald Trump foi o presidente nos últimos 4 anos.',
 'Desta vez, os estatísticos reexaminaram seus modelos, para aumentar o grau de confiabilidade nos seus resultados.',
 'Nesta tentativa de otimização das predições, a inteligência artificial certamente não ficou de fora.',
 'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.',
 'O algoritmo fez a análise de cerca de 1 bilhão de tweets para chegar a uma estimativa dos resultados do pleito.',
 'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.',
 'O Dr. Makse disse que seu trabalho começou já na eleição de 2016, e foi testado novamente nas eleições na Argentina ano pas

In [62]:
melhores_sentencas

['Desta vez, o modelo está treinando com cerca de 5 vezes mais dados que nas eleições americanas anteriores.',
 'Quando seu modelo foi usado para predizer os resultados da eleição corrente usando dados brutos, Joe Biden apareceu como vencedor com larga vantagem.',
 'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.',
 'Segundo o Dr. Makse, integrar estas duas variáveis em seu modelo é a parte mais importante do trabalho.',
 'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.']

In [63]:
notas_sentencas

[(0.10322707053208108,
  'Desta vez, o modelo está treinando com cerca de 5 vezes mais dados que nas eleições americanas anteriores.'),
 (0.08467351062155115,
  'Quando seu modelo foi usado para predizer os resultados da eleição corrente usando dados brutos, Joe Biden apareceu como vencedor com larga vantagem.'),
 (0.07824901512415164,
  'No dia da eleição, 3 de novembro, o modelo estava indicando a vitória de Joe Biden.'),
 (0.07627442688432466,
  'Segundo o Dr. Makse, integrar estas duas variáveis em seu modelo é a parte mais importante do trabalho.'),
 (0.07333552035097592,
  'O modelo desenvolvido pelo Dr. Hernan Makse, físico estatístico da Universidade da Cidade de Nova York, baseou suas predições em uma rede neural treinada para processar os sentimentos expressos nas redes sociais.'),
 (0.07240389589770964,
  'Parece que, desta vez, os algoritmos estão de fato contribuindo para que as predições sejam mais precisas.'),
 (0.06666298135659275,
  'O Dr. Makse disse que seu trabalho 

In [64]:
visualiza_resumo('Teste', sentencas_originais, melhores_sentencas)

In [65]:
import spacy